文本分类涉及【文本表示】和【分类方法】两个问题。。在深度学习的方法出现之前。主流的文本表示为词袋模型 BOW..分类方法为 SVM 和 LR..
对于，一段文本，BOW 表示会忽略其词的顺序，语法和句法。。。将这段文本仅仅看做是一个词的集合。。。因此，BOW 并不能充分表示文本的语义信息。
深度学习模型，克服 BOW 表示的上述缺陷。。它在考虑词顺序的基础上，把文本映射到低纬度的语义空间。。。并且以端对端的方式进行文本表示及分类。。。


对于，一般的短文本分类问题，简单的文本卷积网络。。《短文本处理的方法》


循环神经网络是一种能对序列数据进行精确建模的有力工具。实际上，循环神经网络的理论计算能力是图灵完备的。。自然语言是一种典型的序列数据（词序列）。。目前效果最好的方法。。

//首先，要进行的还是数据预处理。阶段的活儿。。
使用双向循环网络。。
bi-directional recurrent network,
consisting three LSTM layers..
input_dim: here is word dictionary dimension...
emb_dim: dimension of word embedding...
hid_dim: dimension of hidden layer..


feeding用来指定，train_reader 和test_reader 返回的数据与模型配置中的 data_layer 的对应关系。。。现在，知道 word 的数据从哪里来了吧。。。。
name='word'的数据，来自，Reader 返回的第0列数据。。。Reader 返回的第1列数据对应的是 name='label'的 data_layer...

In [ ]:
import sys, os
import paddle.v2 as paddle

with_gpu = os.getenv('WITH_GPU', '0') != '0'


def convolution_net(input_dim, class_dim=2, emb_dim=128, hid_dim=128):
    data = paddle.layer.data("word",
                             paddle.data_type.integer_value_sequence(input_dim))
    emb = paddle.layer.embedding(input=data, size=emb_dim)
    conv_3 = paddle.networks.sequence_conv_pool(
        input=emb, context_len=3, hidden_size=hid_dim)
    conv_4 = paddle.networks.sequence_conv_pool(
        input=emb, context_len=4, hidden_size=hid_dim)
    output = paddle.layer.fc(
        input=[conv_3, conv_4], size=class_dim, act=paddle.activation.Softmax())
    lbl = paddle.layer.data("label", paddle.data_type.integer_value(class_dim))
    cost = paddle.layer.classification_cost(input=output, label=lbl)
    return cost, output


def stacked_lstm_net(input_dim,
                     class_dim=2,
                     emb_dim=128,
                     hid_dim=512,
                     stacked_num=3):
    assert stacked_num % 2 == 1

    fc_para_attr = paddle.attr.Param(learning_rate=1e-3)
    lstm_para_attr = paddle.attr.Param(initial_std=0., learning_rate=1.)
    para_attr = [fc_para_attr, lstm_para_attr]
    bias_attr = paddle.attr.Param(initial_std=0., l2_rate=0.)
    relu = paddle.activation.Relu()
    linear = paddle.activation.Linear()

    data = paddle.layer.data("word",
                             paddle.data_type.integer_value_sequence(input_dim))
    emb = paddle.layer.embedding(input=data, size=emb_dim)

    fc1 = paddle.layer.fc(
        input=emb, size=hid_dim, act=linear, bias_attr=bias_attr)
    lstm1 = paddle.layer.lstmemory(input=fc1, act=relu, bias_attr=bias_attr)

    inputs = [fc1, lstm1]
    for i in range(2, stacked_num + 1):
        fc = paddle.layer.fc(
            input=inputs,
            size=hid_dim,
            act=linear,
            param_attr=para_attr,
            bias_attr=bias_attr)
        lstm = paddle.layer.lstmemory(
            input=fc, reverse=(i % 2) == 0, act=relu, bias_attr=bias_attr)
        inputs = [fc, lstm]

    fc_last = paddle.layer.pooling(
        input=inputs[0], pooling_type=paddle.pooling.Max())
    lstm_last = paddle.layer.pooling(
        input=inputs[1], pooling_type=paddle.pooling.Max())
    output = paddle.layer.fc(
        input=[fc_last, lstm_last],
        size=class_dim,
        act=paddle.activation.Softmax(),
        bias_attr=bias_attr,
        param_attr=para_attr)

    lbl = paddle.layer.data("label", paddle.data_type.integer_value(class_dim))
    cost = paddle.layer.classification_cost(input=output, label=lbl)
    return cost, output


if __name__ == '__main__':
    # init
    paddle.init(use_gpu=with_gpu)

    #data
    print 'load dictionary...'
    import glodonqa
    word_dict = glodonqa.word_dict()
#     word_dict = paddle.dataset.imdb.word_dict()
    dict_dim = len(word_dict)
    import cPickle
    with open("key3_dict.pkl") as f:
        key3_label_dict = cPickle.load(f)
    class_dim = len(key3_label_dict)
#     class_dim = 2
    train_reader = paddle.batch(
        paddle.reader.shuffle(
            glodonqa.train(word_dict), buf_size=1000),
#         paddle.dataset.imdb.train(word_dict), buf_size=1000),
        batch_size=100)
    test_reader = paddle.batch(
        glodonqa.test(word_dict), batch_size=100)
#             paddle.dataset.imdb.test(word_dict), batch_size=100)

    feeding = {'word': 0, 'label': 1}


#     [cost, output] = convolution_net(dict_dim, class_dim=class_dim)
    [cost, output] = stacked_lstm_net(dict_dim, class_dim=class_dim, stacked_num=3)

    # create parameters
    parameters = paddle.parameters.create(cost)

    # create optimizer
    adam_optimizer = paddle.optimizer.Adam(
        learning_rate=2e-3,
        regularization=paddle.optimizer.L2Regularization(rate=8e-4),
        model_average=paddle.optimizer.ModelAverage(average_window=0.5))

    # create trainer
    trainer = paddle.trainer.SGD(
        cost=cost, parameters=parameters, update_equation=adam_optimizer)

    # End batch and end pass event handler
    def event_handler(event):
        if isinstance(event, paddle.event.EndIteration):
            if event.batch_id % 100 == 0:
                print "\nPass %d, Batch %d, Cost %f, %s" % (
                    event.pass_id, event.batch_id, event.cost, event.metrics)
            else:
                sys.stdout.write('.')
                sys.stdout.flush()
        if isinstance(event, paddle.event.EndPass):
            with open('./params_pass_%d.tar' % event.pass_id, 'w') as f:
                trainer.save_parameter_to_tar(f)

            result = trainer.test(reader=test_reader, feeding=feeding)
            print "\nTest with Pass %d, %s" % (event.pass_id, result.metrics)

    # Save the inference topology to protobuf.
    from paddle.v2.plot import Ploter

    train_title = "Train cost"
    cost_ploter = Ploter(train_title)
    step = 0
    def event_handler_plot(event):
        global step
        if isinstance(event, paddle.event.EndIteration):
            cost_ploter.append(train_title, step, event.cost)
            cost_ploter.plot()
            step += 1
    
    
    inference_topology = paddle.topology.Topology(layers=output)
    with open("./inference_topology.pkl", 'wb') as f:
        inference_topology.serialize_for_inference(f)

    trainer.train(
        reader=train_reader,
        event_handler=event_handler,
#         event_handler=event_handler_plot,
        feeding=feeding,
        num_passes=20)


load dictionary...


Building prefix dict from the default dictionary ...
[DEBUG 2018-05-24 15:19:57,129 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
[DEBUG 2018-05-24 15:19:57,137 __init__.py:131] Loading model from cache /tmp/jieba.cache
Loading model cost 0.457 seconds.
[DEBUG 2018-05-24 15:19:57,592 __init__.py:163] Loading model cost 0.457 seconds.
Prefix dict has been built succesfully.
[DEBUG 2018-05-24 15:19:57,597 __init__.py:164] Prefix dict has been built succesfully.
